In [1]:
from bs4 import BeautifulSoup
import unidecode
import re
from sec_downloader import Downloader

import requests
from fake_useragent import UserAgent
import unicodedata
import html

In [2]:
def get_random_headers():
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    return headers

In [27]:
#soup = BeautifulSoup(get_html("AAPL", "10-K"))

C:\Users\310\AppData\Local\Temp\ipykernel_4428\812151145.py:1: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(get_html("AAPL", "10-K"))


In [150]:
url = "https://www.sec.gov/Archives/edgar/data/0000200406/000020040620000010/form10-k20191229.htm"

response = requests.get(url, headers=get_random_headers())
response.raise_for_status()  # Check that the request was successful

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

## Find all the table tags and decompose them to remove from the soup
#for table in soup.find_all('table'):
#    table.decompose()
#
## Now the soup object doesn't contain the tables, you can extract text
#text = soup.get_text(separator=' ', strip=True)

In [151]:
print(soup.title)

<title>Document</title>


In [152]:
text = soup.get_text()
text = unidecode.unidecode(text)
text = re.sub(r'\s+', ' ', text).strip()

print(len(text), text[:1000])


544094 Document false--12-29FY20190000200406P3Y0.0200.0200.0100.0102480000002260000003.323.543.7511432000000043200000003119843000311984300031198430001000000000750000000750000000150000000010000000005000000001000000000750000000750000000150000000010000000005000000000.00250.006500.00890.011250.011500.01650.01650.018750.01950.02050.02250.02450.02450.026250.0290.02950.02950.033750.034000.035000.03550.03550.036250.0370.03750.030.043750.0450.0450.04750.04850.04950.0550.05850.05950.06730.06950.002500.006500.011500.01650.01650.01950.02050.02250.02450.02450.026250.0290.02950.02950.033750.034000.035000.03550.03550.036250.0370.03750.030.043750.0450.0450.04850.04950.0550.05850.05950.06730.0695next 12 months0.211.141.141.141.141.141.26361.10961.10961.10961.10961.10961.2987100000010000002020-09-102019-09-122360000001900000083000000400000022200000019100000070000000270000009600000000002000000200000000P20YP30YP13YP10YP20YP2YP3Y131.94115.6758.65100.0672.54141.06129.5166.07101.8790.44 0000200406 2018-12-31

In [153]:

pattern_mnda = r"(?i)item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*management[’'`]s\s+discussion\s+and\s+analysis\s+of\s+(?:financial\s+condition\s+and\s+results\s+of\s+operations|results\s+of\s+operations\s+and\s+financial\s+condition)"


pattern_qnq = r"(?i)item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*(?:quantitative\s+and\s+qualitative|qualitative\s+and\s+quantitative)?\s+disclosures\s+about\s+market\s+risk"

In [154]:
matches_mnda = re.findall(pattern_mnda, text)
matches_qnq = re.findall(pattern_qnq, text)
print(matches_mnda, '\n', matches_qnq)

["Item 7. Management's Discussion and Analysis of Results of Operations and Financial Condition", "Item 7. Management's Discussion and Analysis of Results of Operations and Financial Condition", "Item 7.MANAGEMENT'S DISCUSSION AND ANALYSIS OF RESULTS OF OPERATIONS AND FINANCIAL CONDITION", "Item 7. Management's Discussion and Analysis of Results of Operations and Financial Condition", "Item 7. Management's Discussion and Analysis of Results of Operations and Financial Condition"] 
 ['Item 7A.QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK']


In [155]:
matches_mnda = re.finditer(pattern_mnda, text)
matches_qnq = re.finditer(pattern_qnq, text)

for i, mnda in enumerate(matches_mnda):
    print(f'mnda: {mnda.end()}')
for x, qnq in enumerate(matches_qnq):
    print(f'qnq: {qnq.start()}')
    
    
    #print(f'mnda №{i+1}:{mnda.span()} qnq №{x+1}: {qnq.span()}')

mnda: 122922
mnda: 171395
mnda: 189250
mnda: 193988
mnda: 254360
qnq: 254121


In [156]:
import re

def find_longest_substring(text, pattern_mnda, pattern_qnq):
    matches_mnda = list(re.finditer(pattern_mnda, text))
    matches_qnq = list(re.finditer(pattern_qnq, text))
    
    longest_substring = ""
    
    for mnda_match in matches_mnda:
        mnda_end = mnda_match.end()
        # Find the first matches_qnq after this mnda_match
        for qnq_match in matches_qnq:
            if qnq_match.start() > mnda_end:
                substring = text[mnda_end:qnq_match.start()]
                if len(substring) > len(longest_substring):
                    longest_substring = substring
                break  # We only want the first qnq match after each mnda match
    
    return longest_substring

# Example usage
longest = find_longest_substring(text, pattern_mnda, pattern_qnq)
print(f"The longest substring found is: '{len(longest)}'")


The longest substring found is: '131199'


In [160]:
import re

# Assuming 'text' is the variable containing the text extracted from your document
text = """
Item 2 — MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Item 10.    Management’s Discussion and Analysis of Financial Condition and Results of Operations

Item 2A.    Management's Discussion and Analysis of Financial Condition and Results of Operations.

Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations

ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Item 7.	
MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS

Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations

Item 7.
MANAGEMENT’S DISCUSSION AND ANALYSIS OF RESULTS OF OPERATIONS AND FINANCIAL CONDITION

"""

text = unidecode.unidecode(text)
text = re.sub(r'\s+', ' ', text).strip()

print(len(text), text)





756 Item 2 -- MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Item 10. Management's Discussion and Analysis of Financial Condition and Results of Operations Item 2A. Management's Discussion and Analysis of Financial Condition and Results of Operations. Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS Item 7. Management's Discussion and Analysis of Financial Condition and Results of Operations Item 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF RESULTS OF OPERATIONS AND FINANCIAL CONDITION


In [162]:

#pattern_mnda = r"(?i)item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*management[’'`]s\s+discussion\s+and\s+analysis\s+of\s+financial\s+condition\s+and\s+results\s+of\s+operations"
#test for either order
pattern_mnda = r"(?i)^\s*item\s+\d+[A-Za-z]?\.?\s*(?:--|[-—])?\s*management[’'`]s\s+discussion\s+and\s+analysis\s+of\s+(?:financial\s+condition\s+and\s+results\s+of\s+operations|results\s+of\s+operations\s+and\s+financial\s+condition)\s*$"

matches = re.findall(pattern_mnda, text, re.MULTILINE)
print(matches)
print('\n')
print(len(matches) == 8)
print(len(matches))

[]


False
0
